In [46]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import shapely.geometry, shapely.ops
import fiona, fiona.crs
import pyproj
from pprint import pprint

from __future__ import print_function, division

# Point contained by shapefile?

See this Stackoverflow question: http://stackoverflow.com/questions/7861196/check-if-a-geopoint-with-latitude-and-longitude-is-within-a-shapefile.

TODO: Initial bounding-box check.

In [61]:
fname = 'data/naturalearth/110m/cultural/ne_110m_admin_0_countries.shp'
with fiona.open(fname) as c:
    r = next(iter(c))
    shape = shapely.geometry.asShape(r['geometry'])
    point = shapely.geometry.Point(67.72, 34.64)
    
    #print(shape.contains(point))
    #print(shape.bounds)
    #print(fiona.crs.to_string(c.crs))
    p = pyproj.Proj(fiona.crs.to_string(c.crs))
    shape = shapely.ops.transform(p, shape)
    point = shapely.ops.transform(p, point)
    
    
    print(point)
    print(shape)
    print(shapely.geometry.box(*shape.bounds).contains(point))
    print(shape.contains(point))

POINT (1.18193696945056 0.6045820528908358)
POLYGON ((1.068330296086634 0.6222111407903833, 1.086129764039545 0.6155892155960451, 1.099289735506528 0.6179170811540112, 1.102935311866582 0.6258256007762607, 1.116712199273762 0.6284574146848796, 1.126548581209128 0.6337652367726867, 1.130032713193019 0.6477234108815917, 1.144743091825497 0.651098861036837, 1.147477725057483 0.657311312785489, 1.15571138824178 0.6526447585828867, 1.160968702591794 0.6521036042493804, 1.170693245964901 0.6519877070296212, 1.183856825127441 0.6483024460184433, 1.189189901084146 0.6461752585258193, 1.201824050923738 0.6517816174209442, 1.207702790833395 0.6484097749612556, 1.213331697825749 0.6564008206193647, 1.22376515337575 0.6560382472159156, 1.226452886565498 0.6586028677947905, 1.228298222842754 0.6656405799020397, 1.235812150763488 0.67171233152398, 1.245259803502619 0.6677438664116006, 1.24336215563979 0.6624094375690627, 1.248642018087033 0.6615805695149087, 1.247014947390957 0.646917541886612, 1.25

In [62]:
def point_in_shape(c, r, lon, lat):
    shape = shapely.geometry.asShape(r['geometry'])
    point = shapely.geometry.Point(lon, lat)
    
    # Project both shape and point into preferred system
    p = pyproj.Proj(fiona.crs.to_string(c.crs))
    shape = shapely.ops.transform(p, shape)
    point = shapely.ops.transform(p, point)
    
    if shapely.geometry.box(*shape.bounds).contains(point):
        if shape.contains(point):
            return True
    
    return False

In [64]:
fname = 'data/naturalearth/110m/cultural/ne_110m_admin_0_countries.shp'
with fiona.open(fname) as c:
    #r = next(iter(c))
    r = c[10]
    
    lon, lat = 67.72, 34.64
    print(point_in_shape(c, r, lon, lat))

False


In [73]:
class GeoShape(object):
    def __init__(self, fname, idx):
        with fiona.open(fname) as c:
            r = c[idx]
            self.proj = pyproj.Proj(fiona.crs.to_string(c.crs))
            shape = shapely.geometry.asShape(r['geometry'])
            self.shape = shapely.ops.transform(p, shape)
            self.bounds = shapely.geometry.box(*self.shape.bounds)
    
    def contains(self, lon, lat):
        point = shapely.geometry.Point(lon, lat)
        point = shapely.ops.transform(p, point)
        
        if self.bounds.contains(point):
            if self.shape.contains(point):
                return True
        
        return False

In [74]:
gs = GeoShape(fname, 0)

In [82]:
gs.contains(67.7-360., 1.+34.6)

True